In [ ]:
import os
import getpass

key = os.environ.get("FRED_API_KEY")
if not key:
    key = getpass.getpass("Insert FRED API key (input hidden): ")
    os.environ["FRED_API_KEY"] = key

print("FRED_API_KEY set:", "YES" if os.environ.get("FRED_API_KEY") else "NO")


In [ ]:
import subprocess, sys
from pathlib import Path

PROJECT_ROOT = Path.cwd()  # odpal notebook z folderu projektu!
SRC = PROJECT_ROOT / "src"

def run_module(module_name: str):
    """Run src/<module_name>.py as python -m src.<module_name>."""
    cmd = [sys.executable, "-m", f"src.{module_name}"]
    res = subprocess.run(cmd, cwd=PROJECT_ROOT, capture_output=True, text=True)
    print(res.stdout)
    if res.stderr:
        print(res.stderr)
    if res.returncode != 0:
        raise RuntimeError(f"Module {module_name} failed (code={res.returncode}).")


In [ ]:
# --- Data download ---
run_module("data_download")              # equity
run_module("data_download_fx")           # FX
run_module("data_download_bonds_fred")   # bonds (FRED)
run_module("data_download_commodities")  # commodities

# --- Rolling skewness ---
run_module("skewness")
run_module("skewness_fx")
run_module("skewness_bonds")
run_module("skewness_commodities")

# --- Portfolios ---
run_module("portfolios")
run_module("portfolios_fx")
run_module("portfolios_bonds")
run_module("portfolios_commodities")

# --- Analysis ---
run_module("analysis_equity")
run_module("analysis_fx")
run_module("analysis_bonds")
run_module("analysis_commodities")

# --- Regressions ---
run_module("regression_equity")
run_module("regression_bonds")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# =========================
# Paths / output
# =========================
DATA_PROCESSED = Path("data") / "processed"
OUT_DIR = Path("outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# =========================
# Helpers
# =========================
def cumret(x: pd.Series) -> pd.Series:
    x = pd.Series(x).dropna()
    return (1.0 + x).cumprod()

def plot_cum(series_dict, title, outname=None):
    plt.figure()
    for label, s in series_dict.items():
        s = pd.Series(s).dropna()
        plt.plot(cumret(s), label=label)
    plt.title(title)
    plt.xlabel("Date")
    plt.ylabel("Cumulative growth (1 + r)")
    plt.legend()
    plt.tight_layout()
    if outname:
        plt.savefig(OUT_DIR / outname, dpi=200)
    plt.show()

def read_pf(path: Path) -> pd.DataFrame:
    pf = pd.read_csv(path, index_col=0, parse_dates=True).sort_index()
    if "LongShort" not in pf.columns:
        raise ValueError(f"{path} nie ma kolumny 'LongShort'.")
    return pf

# =========================
# Load inputs from disk
# =========================
equity_ret = pd.read_csv(
    DATA_PROCESSED / "equity_monthly_returns.csv",
    index_col=0,
    parse_dates=True,
).sort_index()

pf_eq  = read_pf(DATA_PROCESSED / "portfolio_skewness_ls.csv")
pf_fx  = read_pf(DATA_PROCESSED / "portfolio_fx_skewness_ls.csv")
pf_bd  = read_pf(DATA_PROCESSED / "portfolio_bonds_skewness_ls.csv")
pf_cmd = read_pf(DATA_PROCESSED / "portfolio_commodities_skewness_ls.csv")

print("Loaded:")
print(" equity_ret:", equity_ret.shape)
print(" pf_eq:", pf_eq.shape)
print(" pf_fx:", pf_fx.shape)
print(" pf_bd:", pf_bd.shape)
print(" pf_cmd:", pf_cmd.shape)

# =========================================================
# 1) EQUITIES — jedyne miejsce z benchmarkiem rynkowym
# =========================================================
mkt_eq = equity_ret.mean(axis=1)  # equal-weight market
idx_eq = pf_eq.index.intersection(mkt_eq.index)

plot_cum(
    {
        "Market EW": mkt_eq.loc[idx_eq],
        "Skewness LS": pf_eq.loc[idx_eq, "LongShort"],
    },
    title="Equities — Market EW vs Skewness Long-Short",
    outname="equities_cum_market_vs_skewls.png",
)

# ==========================================
# 2) FX — tylko Skewness LS
# ==========================================
plot_cum(
    {"FX Skewness LS": pf_fx["LongShort"]},
    title="FX — Skewness Long-Short",
    outname="fx_cum_skewls.png",
)

# ==========================================
# 3) BONDS — tylko Skewness LS
# ==========================================
plot_cum(
    {"Bonds Skewness LS": pf_bd["LongShort"]},
    title="Bonds — Skewness Long-Short",
    outname="bonds_cum_skewls.png",
)

# ==============================================
# 4) COMMODITIES — tylko Skewness LS
# ==============================================
plot_cum(
    {"Commodities Skewness LS": pf_cmd["LongShort"]},
    title="Commodities — Skewness Long-Short",
    outname="commodities_cum_skewls.png",
)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

OUT_DIR = Path("outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 1) Ustawiamy dokładną kolejność jak w pracy
order = ["Equity_LS", "FX_LS", "Bond_LS", "Cmd_LS"]
corr = mean_roll_corr.loc[order, order].copy()

fig, ax = plt.subplots(figsize=(7.2, 5.6))

im = ax.imshow(
    corr.values,
    vmin=-1, vmax=1,
    cmap="coolwarm",          # jak w pracy (diverging)
    interpolation="nearest",  # "kafelki", bez rozmycia
    aspect="equal"
)

ax.set_title("Mean 36-Month Rolling Correlation — Skewness LS Strategies", pad=10)

ax.set_xticks(np.arange(len(order)))
ax.set_yticks(np.arange(len(order)))
ax.set_xticklabels(["Equity LS", "FX LS", "Bond LS", "Cmd LS"], fontsize=11)
ax.set_yticklabels(["Equity_LS", "FX_LS", "Bond_LS", "Cmd_LS"], fontsize=11)

# 2) Delikatna siatka (jak na screenie)
ax.set_xticks(np.arange(-.5, len(order), 1), minor=True)
ax.set_yticks(np.arange(-.5, len(order), 1), minor=True)
ax.grid(which="minor", color="black", linestyle="-", linewidth=1, alpha=0.35)
ax.tick_params(which="minor", bottom=False, left=False)

# 3) Liczby w komórkach (format jak u Ciebie: 3 miejsca dla małych)
for i in range(corr.shape[0]):
    for j in range(corr.shape[1]):
        v = float(corr.iloc[i, j])
        if abs(v) < 0.01 and v != 0:
            txt = f"{v:.5f}".rstrip("0").rstrip(".")
        else:
            txt = f"{v:.3f}".rstrip("0").rstrip(".")
        ax.text(j, i, txt, ha="center", va="center", fontsize=10, color="black")

# 4) Colorbar
cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label("Correlation", fontsize=11)
cbar.ax.tick_params(labelsize=10)

# 5) Kosmetyka jak w pracy
for spine in ax.spines.values():
    spine.set_visible(False)

plt.tight_layout()
plt.savefig(OUT_DIR / "mean_36m_rolling_corr_heatmap_like_thesis.png", dpi=250)
plt.show()
